In [ ]:
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix as cm
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.models import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image, ImageEnhance

# For Data Visualization      
import matplotlib.pyplot as plt
import seaborn as sns

# Miscellaneous
from tqdm import tqdm
import time
import os
import random
import pandas as pd
from IPython import display


In [ ]:
data_dir = '/kaggle/input/binary-cropped-crown-of-thorns-dataset/'

unique_labels = os.listdir(data_dir)
all_paths = []
all_labels = []

for label in os.listdir(data_dir):
    for image in os.listdir(data_dir+label):
        all_paths.append(data_dir+label+'/'+image)
        all_labels.append(label)

all_paths, all_labels = shuffle(all_paths, all_labels)
x_train_paths, x_val_paths, y_train, y_val = train_test_split(all_paths, all_labels, test_size=0.2, random_state=42)

In [ ]:
base_model = MobileNetV2(input_shape=(128, 128, 3), weights='imagenet', include_top=False)

In [ ]:
def augment_image(image):
    if random.uniform(0,1)>0.5:
        image = np.fliplr(image)
    image = Image.fromarray(np.uint8(image))
    image = ImageEnhance.Brightness(image).enhance(random.uniform(0.6,1.4))
    image = ImageEnhance.Contrast(image).enhance(random.uniform(0.6,1.4))
    image = ImageEnhance.Color(image).enhance(random.uniform(0.6,1.4))
    image = ImageEnhance.Sharpness(image).enhance(random.uniform(0.6,1.4))
    return image
def open_images(paths, augment=True):
    images = []
    for path in paths:
        image = load_img(path, target_size=(128,128))
        image = np.array(image)
        if augment:
            image = augment_image(image)
        image = np.array(image)/255.0
        images.append(image)
    return np.array(images)

def encode_label(labels):
    encoded = []
    for x in labels:
        encoded.append(unique_labels.index(x))
    return np.array(encoded)

def decode_label(labels):
    decoded = []
    for x in labels:
        decoded.append(unique_labels[x])
    return np.array(decoded)


In [ ]:
x=open_images(x_train_paths[0:7000])
x.shape
y=encode_label(y_train[0:7000])

In [ ]:
model = Sequential()
model.add(Input(shape=(128,128,3)))
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.25))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(len(unique_labels),activation='softmax'))
model.summary()
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
len(data_dir)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd


train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.000175),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])
history = model.fit(
    x_train,y_train,
    validation_data=(x_val,y_val),
    batch_size=32,
    epochs=15  
)

In [ ]:
epochs=15
plt.figure(figsize=(8,8))
plt.grid(True)
plt.plot(history.history['val_sparse_categorical_accuracy'], '.g-', linewidth=2)
plt.plot(history.history['loss'], '.r-', linewidth=2)
plt.title('Model Training History')
plt.xlabel('epoch')
plt.xticks([x for x in range(epochs)])
plt.legend(['Accuracy', 'Loss'], loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input
cots='../input/binary-cropped-crown-of-thorns-dataset/cots_crops/cotscrop-video_0-1000-0.jpg'
not_cots='../input/newdataset/pic8.png'
def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

def make_predictions(image_path):
    show_image(image_path)
    image = image_utils.load_img(image_path, target_size=(128, 128))
    image = image_utils.img_to_array(image)
    image = image.reshape(1, 128, 128, 3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds
    


In [ ]:
show_image(cots)
make_predictions(cots)

In [ ]:
show_image(not_cots)
make_predictions(not_cots)